In [ ]:
################################################
# New style 
################################################
import sys
import os

workdir_ = '/glade/work/juliob'
if ( workdir_ not in sys.path ):
    sys.path.append(workdir_)
    print( f" a path to {workdir_} added in {__name__} ")



# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature


from myPythonTools.Utils import utils as uti
from myPythonTools.Utils import numerical_utils as nuti
from myPythonTools.Utils import AveragingUtils as Av
from myPythonTools.Utils import validation_data as Val
from myPythonTools.Utils import PlotUtil as Pu
from myPythonTools.Plotting import LatLonMaps as LL 
from myPythonTools.Plotting import utils as plu
from myPythonTools.CASutils import filter_utils as fu 

from PyRegridding.Utils import GridUtils as GrU
from PyRegridding.Utils import MakePressures as MkP
from PyRegridding.Drivers import RegridField as RgF


# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( uti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
#importlib.reload(vAB)
importlib.reload(MkP)
importlib.reload(RgF)


In [ ]:
print( sys.path )
############################
# Initialize dict for regridding
############################
RgObs={}
yamldir='../YAML/'

In [ ]:
### Make a list of files
import yaml
#from box import Box


with open(f'{yamldir}configure_plotting.yaml', 'r') as file:
    cfgPlot = yaml.safe_load(file)


######################################################
season = 'sep' #cfgPlot['zonal_mean']['season']
valData  = cfgPlot['zonal_mean']['validation']['data']
valYears = cfgPlot['zonal_mean']['validation']['years']
print( f"Season= {season}" )
print( valData , valYears )
fld='Nudge_U'

In [ ]:

#"""
exp_file=f'{yamldir}GW_ndg_ppe.yaml'
xl=['ndg00','ndg01c','ndg02b']
xl=['ndg00','ndg01c','ndg01d']
#"""


with open( exp_file, 'r') as file:
    cfg = yaml.safe_load(file)

Al =[]
for x in xl:
    exp, subd, Src, Hkey, Dst, useri = cfg[x]['name'] , cfg[x]['subdir'] , cfg[x]['SrcGrid'] , cfg[x]['Hkey'] , cfg[x]['DstGrid'] , cfg[x]['user'] 
    ymdPat = '2014-07-*' #cfg[x]['ymdPat']
    print( exp, subd, Src, Hkey, Dst, useri )
    print( ymdPat  )
    
    A = uti.MakeDict4Exp( exp=exp , user=useri, subd=subd , 
                     hsPat='cam.h2a' , ymdPat=ymdPat,verbose=True, open_dataset=True )
    A['Src']=Src
    A['Hkey']=Hkey
    if 'gw_parameters' in cfg[x]:
        A['gw_parameters'] = cfg[x]['gw_parameters']
    Al.append( A )
    print( f'{A.exp} has {len(A.X.time)} times  \n')


In [ ]:
Al[1].X

In [ ]:
########################################
# Make seasonal means
########################################

for A in Al:

    A['utgw']=A.X.UTEND_GWDTOT.values
    A['utndg']=A.X.Nudge_U.values
    A['utphys']=A.X.UTEND_PHYSTOT.values
    if ('UTGW_MOVMTN' in A.X):
        print( f'UTGW_MOVMTN found in {A.exp}' )
        A['utgw_mm']=A.X.UTGW_MOVMTN.values
        
    #A['years']=years_a
    #A['months']=months_a


In [ ]:
A.utphys.shape

In [ ]:

utphys=86_400.* Al[1].utphys[:,10,:]
utndg=86_400.*A.utndg[:,10,:]
utgw=86_400.*A.utgw[:,10,:]


In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(28, 5))

# Flatten axs to easily access each subplot by index
ax = ax.flatten()
ax[0].plot( np.average( (86_400.*Al[1].utphys[:,10,:])**2, axis=0 )**0.5 ,'.')
ax[1].plot( np.average( (86_400.*Al[2].utphys[:,10,:])**2, axis=0 )**0.5 ,'.')

ax[2].plot( np.average( (86_400.*Al[1].utndg[:,10,:])**2, axis=0 )**0.5 ,'.')
ax[3].plot( np.average( (86_400.*Al[2].utndg[:,10,:])**2, axis=0 )**0.5 ,'.')


#ax[1].plot( np.average( (86_400.*Al[1].utndg[:,10,:])**2, axis=0 )**0.5 ,'.')
#ax[2].plot( np.average( (86_400.*Al[1].utgw[:,10,:])**2, axis=0 )**0.5 ,'.')


In [ ]:
t,z=10,25

fig, ax = plt.subplots(1, 4, figsize=(28, 5))

# Flatten axs to easily access each subplot by index
ax = ax.flatten()
ax[0].plot( (86_400.*Al[1].utphys[t,z,:]) ,'.')
ax[0].plot( (86_400.*Al[1].utgw[t,z,:]) ,'.')

ax[1].plot( (86_400.*Al[2].utphys[t,z,:]) ,'.')
ax[1].plot( (86_400.*(Al[2].utgw+Al[2].utndg)[t,z,:]) ,'.')


ax[2].plot( (86_400.*Al[1].utndg[t,z,:])  ,'.')
ax[3].plot( (86_400.*Al[2].utndg[t,z,:]) ,'.')


#ax[1].plot( np.average( (86_400.*Al[1].utndg[:,10,:])**2, axis=0 )**0.5 ,'.')
#ax[2].plot( np.average( (86_400.*Al[1].utgw[:,10,:])**2, axis=0 )**0.5 ,'.')


In [ ]:

plt.scatter( (Al[0].utgw-Al[1].utgw )[:,20,:].flatten() , (Al[0].utndg-Al[1].utndg )[:,20,:].flatten() , s=.1 )


In [ ]:

#plt.scatter( (Al[0].utgw[:,20,:]-Al[1].utgw[1:,20,:]).flatten() , (Al[0].utndg[:,20,:]-Al[1].utndg[1:,20,:]).flatten() , s=.1 )


In [ ]:
%%time

########################################
# Make zonal means
########################################
method='C2'

for A in Al:
    Hkey = A['Hkey']
    Src  = A['Src']
    utgw  = A['utgw']
    utndg = A['utndg']
    if ('utgw_mm' in A):
        utgw_mm  = A['utgw_mm']
        
    
    
    if (Hkey == 'yx' ):
        ########################
        # Option 1 from lat-lon regirrdded data
        ########################

        lon_a=A.X.lon.values
        lat_a=A.X.lat.values
        lev_a=A.X.lev.values
        zlev_a=-7.0*np.log( lev_a/1_000. )
        utgw_yx  =utgw
        utndg_yx =utndg
        if ('utgw_mm' in A):
            utgw_mm_yx   = utgw_mm
    
    elif (Hkey=='c'):
        ########################
        # Option 2 from unstructured SE data
        ########################
        
        Dst = 'fv1x1'
        DstInfo = GrU.gridInfo(Dst) #,Vgrid=DstVgrid)
        lat_a,lon_a = GrU.latlon( scrip= DstInfo['scrip'], Hkey=DstInfo['Hkey'] )
        lev_a=A.X.lev.values
        zlev_a=-7.0*np.log( lev_a/1_000. )
    
        RegridObj = GrU.regrid_object_lib(RgOb=RgObs, src=Src, dst=Dst)
        utgw_yx   = RgF.Horz(xfld_Src=utgw  , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
        utndg_yx  = RgF.Horz(xfld_Src=utndg , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
        if ('utgw_mm' in A):
            utgw_mm_yx   = RgF.Horz(xfld_Src=utgw_mm  , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
        
    A['utgw_yx']  = utgw_yx
    A['utndg_yx'] = utndg_yx
    if ('utgw_mm' in A):
        A['utgw_mm_yx']  = utgw_mm_yx
    A['lonx'] = lon_a
    A['latx'] = lat_a
    A['zlev'] = zlev_a
    A['plev'] = lev_a
    A['DateRange'] = f"{(A.X.time[0].values.item() ).strftime('%Y-%m-%d')}_{(A.X.time[-1].values.item() ).strftime('%Y-%m-%d')}"

In [ ]:
lev=np.linspace(-1.,1,51)
# Create a figure with a grid of subplots
fig, ax = plt.subplots(1, 3, figsize=(20, 5))

# Flatten axs to easily access each subplot by index
ax = ax.flatten()

z=15
print( Al[0].zlev[z])

co=ax[0].contourf(  86_400* np.average( (Al[0].utndg_yx )[:,z,:,:], axis=0 ) , levels=6*lev , cmap='bwr' )
cb=plt.colorbar( co )
co=ax[1].contourf(  86_400* np.average( (Al[1].utgw_mm_yx )[:,z,:,:], axis=0 ) , levels=lev , cmap='bwr' )
cb=plt.colorbar( co )
co=ax[2].contourf(  86_400* np.average( (Al[2].utgw_mm_yx )[:,z,:,:], axis=0 ) , levels=lev , cmap='bwr' )
cb=plt.colorbar( co )

#co=ax[2].contourf( 86_400* Al[0].utgw_yx[:,t,z,:] , levels=0.5*lev , cmap='bwr' ) 
#cb=plt.colorbar( co )



In [ ]:
lev=np.linspace(-1.,1,51)

z=15
print( Al[0].zlev[z])


fields=[ 86_400* np.average( (Al[0].utndg_yx )[:,z,:,:], axis=0 ) , 86_400* np.average( (Al[1].utgw_mm_yx )[:,z,:,:], axis=0 ) , 86_400* np.average( (Al[2].utgw_mm_yx )[:,z,:,:], axis=0 ) ]
lons=[  Al[0].lonx,  Al[0].lonx,  Al[0].lonx, ]
lats=[  Al[0].latx,  Al[0].latx,  Al[0].latx, ]
clevs = [ 6*lev, lev,lev ]
cmaps=['bwr','bwr','bwr',]

####
MapProj = ccrs.PlateCarree(central_longitude=180.)

#LL.Maps( fields=fields ,lons=lons ,lats=lats ,clevs=clevs,cmaps=cmaps , titles=titles, insets=insets, Projection=MapProj, Landmask=False )

LL.Maps( fields=fields ,lons=lons ,lats=lats ,clevs=clevs,cmaps=cmaps , Projection=MapProj, Landmask=False )


In [ ]:
plt.contour( (Al[1].utndg_yx -Al[0].utndg_yx)[100,20,:,:] )

In [ ]:
plt.plot(  (Al[0].utgw_yx-Al[1].utgw_yx )[:,35,50,:].flatten() ) 
#plt.plot(  (Al[0].utndg_yx-Al[1].utndg_yx )[:,30,50,100].flatten() )
plt.plot(  (Al[0].utndg_yx )[:,35,50,:].flatten() )


In [ ]:
lev=np.linspace(-10.,10,51)
# Create a figure with a grid of subplots
fig, ax = plt.subplots(1, 3, figsize=(20, 5))

# Flatten axs to easily access each subplot by index
ax = ax.flatten()

z=20
y=40

co=ax[0].contourf( 86_400* (Al[1].utgw_yx-0*Al[0].utgw_yx )[:,z,y,:] , levels=0.5*lev , cmap='bwr' ) 
cb=plt.colorbar( co )

co=ax[1].contourf(  86_400* (Al[0].utndg_yx )[:,z,y,:] , levels=lev , cmap='bwr' )
cb=plt.colorbar( co )

co=ax[2].contourf( 86_400* Al[0].utgw_yx[:,z,y,:] , levels=0.5*lev , cmap='bwr' ) 
cb=plt.colorbar( co )




In [ ]:
print(Al[0].zlev[20])

In [ ]:
plt.scatter( (Al[0].utgw_yx-Al[1].utgw_yx )[:,15,20:70,:].flatten() , (Al[0].utndg_yx-Al[1].utndg_yx )[:,15,20:70,:].flatten() , s=.1 )
#plt.xlim(-0.0004,0.0004)
plt.xlim(-0.0001,0.0001)
plt.ylim(-0.00003,0.00003)


In [ ]:
plt.scatter( (Al[0].utgw_yx-Al[1].utgw_yx )[:,15,20:70,:].flatten() , Al[0].utndg_yx[:,15,20:70,:].flatten() , s=.1 )
#plt.xlim(-0.0004,0.0004)
#plt.xlim(-0.0001,0.0001)


In [ ]:
plt.scatter( (Al[0].utgw_yx-Al[1].utgw_yx )[:,15,20:70,:].flatten() , (Al[0].utndg_yx-Al[1].utndg_yx )[:,15,20:70,:].flatten() , s=.1 )
#plt.xlim(-0.0004,0.0004)
plt.xlim(-0.0001,0.0001)
plt.ylim(-0.00003,0.00003)


In [ ]:
####
# Validation
####
Validation_on_control_mgrid = False

if (Validation_on_control_mgrid == True):
    PSc_s = Av.Seasonal( ds=Dc, season=season , fld='PS')
    mgrid={'ps':PSc_s, 'hyam':hyam_c, 'hybm':hybm_c, 'hyai':hyai_c, 'hybi':hybi_c, 'hgrid':'fv0.9x1.25' }
else:
    mgrid=False


valYears='2014' #'*'
Fval = Val.data(fld=fld,season=season,mgrid=mgrid,zlev=True , Years=valYears ) #, Years='1996' ) #, Years='2000' )
if (Fval['rcode'] == 0):
    Fv_s , zlev_v,lat_v,lon_v, years_Uv = Fval['aa'], Fval['lev'], Fval['lat'], Fval['lon'], Fval['years']
    Fv_sz=np.average( Fv_s , axis=2 )



In [ ]:


#nxplo,nyplo=4,1
#fig = plt.figure(figsize=(24, 5))

numx=len( Al )
nxplo,nyplo=numx+1,2
fig = plt.figure(figsize=(8*nxplo, 18))

flev,dlev,scale= plu.contour_intervals( fld , zonal_mean=1 )
cmap='gist_ncar'

titlesize=18

plotTop=65 #85
BoxParm =['alpha_gw_movmtn', 'effgw_rdg_resid','use_gw_rdg_resid']
BoxParm =['use_gw_convect_dp','use_gw_movmtn_pbl','use_gw_front','use_gw_rdg_resid','use_gw_rdg_beta','use_gw_oro']
BoxParm =['use_gw_movmtn_pbl','use_gw_rdg_resid','movmtn_source','alpha_gw_movmtn', 'effgw_movmtn_pbl','effgw_rdg_resid',]

ix=0

for A in Al:
    
    Fa_sz = A['F_sz'] 
    lon_a = A['lonx']
    lat_a = A['latx']
    zlev_a = A['zlev']
    years_a = A['years']
    if 'gw_parameters' in A:
        parameters=A['gw_parameters']
    else:
        parameters=None

    # Create 2D arrays
    Lat_a, Zlev_a = np.meshgrid(lat_a, zlev_a)
    
    # Adding the hatching for the sponge layer
    sponge_layer = np.zeros_like(Fa_sz)
    sponge_layer[(Zlev_a >= 62) & (Zlev_a <= 85)] = 1
    
    title = f"<{A.exp}> \n {season.upper()} {years_a[0]}-{years_a[-1]}"
    n=ix+1
    Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo, ypan=0.8 ) 
    ax1 = fig.add_axes( Axes1 )
    co1=ax1.contourf(lat_a,zlev_a, scale*Fa_sz ,levels=flev, cmap=cmap )
    co2=ax1.contour(lat_a,zlev_a,  scale*Fa_sz ,levels=flev, colors='black')
    #ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
    ax1.set_title( title ,fontsize=titlesize )
    ax1.set_ylim(0,plotTop)
    cb=plt.colorbar(co1)
    ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
        fontsize=20, fontweight='bold', va='top')
    
    # Sponge layer hatching
    ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)
    
    Pu.param_text_box( ax=ax1, parameters=parameters, select=BoxParm ) 



    ix=ix+1
    
if (Fval['rcode'] == 0):
    for ix in np.arange( start=numx,stop=numx+1 ):
    
        title = f"<{valData}> {season.upper()} {years_Uv} " 
        n=ix+1
        Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo, ypan=0.8 ) 
        ax1 = fig.add_axes( Axes1 )
        co1=ax1.contourf(lat_v,zlev_v,scale*Fv_sz ,levels=flev, cmap=cmap )
        co2=ax1.contour(lat_v,zlev_v, scale*Fv_sz ,levels=flev, colors='black')
        ax1.set_title( title ,fontsize=titlesize )
        ax1.set_ylim(0,plotTop)
        cb=plt.colorbar(co1)
        ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
            fontsize=20, fontweight='bold', va='top')

FigName = f"multiFig.png" #f"{A.exp}_{season.upper()}_{years_a[0]}-{years_a[-1]}_Dst{Dst}_zUT_climo.png"
print( FigName) 
Pu.save_link(png_filename = FigName )

